In [36]:
import pandas as pd
import numpy as np
import csv
import re


In [37]:
csv_path = '/Users/amulya/Desktop/Elsewhere/me/Work/Hackathons/Numhack/Temperature-and-Coral-Bleaching/Data/coral.csv'


In [38]:
# Read the CSV file
try:
    df = pd.read_csv(
        csv_path,
        na_values=['nd'],       # Replace 'nd' with NaN
        keep_default_na=True,   # Keep default NaN values
        quotechar='"',          # Character used to denote the start and end of a quoted item
        escapechar='\\',        # Character used to escape the quotechar inside a quoted item
        encoding='utf-8',       # Ensure proper encoding
        engine='python',        # Use the Python engine for better flexibility with malformed files
        on_bad_lines='warn'     # Warn about bad lines
    )
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error loading CSV file: {e}")

CSV file loaded successfully.


In [39]:
print("Initial DataFrame:")
df.head()

Initial DataFrame:


,Site_ID,Sample_ID,Data_Source,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Reef_ID,Realm_Name,Ecoregion_Name,Country_Name,...,TSA_FrequencyMax,TSA_FrequencyMean,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean,Date,Site_Comments,Sample_Comments,Bleaching_Comments
0,2501,10324336,Donner,23.16,-82.53,Atlantic,NaN,Tropical Atlantic,Cuba and Cayman Islands,Cuba,...,5.00,0.00,0.00,0.74,7.25,0.18,2005-09-15,NaN,NaN,NaN
1,3467,10324754,Donner,-17.57,-149.78,Pacific,NaN,Eastern Indo-Pacific,Society Islands French Polynesia,French Polynesia,...,4.00,0.00,0.26,0.67,4.65,0.19,1991-03-15,The bleaching does not appear to have gained ...,The bleaching does not appear to have gained ...,NaN
2,1794,10323866,Donner,18.37,-64.56,Atlantic,NaN,Tropical Atlantic,Hispaniola Puerto Rico and Lesser Antilles,United Kingdom,...,7.00,0.00,0.00,1.04,11.66,0.26,2006-01-15,NaN,NaN,NaN
3,8647,10328028,Donner,17.76,-64.57,Atlantic,NaN,Tropical Atlantic,Hispaniola Puerto Rico and Lesser Antilles,United States,...,4.00,0.00,0.00,0.75,5.64,0.20,2006-04-15,NaN,NaN,NaN
4,8648,10328029,Donner,17.77,-64.58,Atlantic,NaN,Tropical Atlantic,Hispaniola Puerto Rico and Lesser Antilles,United States,...,5.00,0.00,0.00,0.92,6.89,0.25,2006-04-15,NaN,NaN,NaN


In [40]:
print("\nDataFrame Info:")
df.info()


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41361 entries, 0 to 41360
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Site_ID                                41361 non-null  int64  
 1   Sample_ID                              41361 non-null  int64  
 2   Data_Source                            41361 non-null  object 
 3   Latitude_Degrees                       41361 non-null  float64
 4   Longitude_Degrees                      41361 non-null  float64
 5   Ocean_Name                             41361 non-null  object 
 6   Reef_ID                                28821 non-null  object 
 7   Realm_Name                             41361 non-null  object 
 8   Ecoregion_Name                         41358 non-null  object 
 9   Country_Name                           41360 non-null  object 
 10  State_Island_Province_Name             41262 non-null

In [41]:
print("\nMissing Values in Each Column:")
df.isna().sum()


Missing Values in Each Column:


Site_ID                   0
Sample_ID                 0
Data_Source               0
Latitude_Degrees          0
Longitude_Degrees         0
                      ...  
TSA_DHWMean             132
Date                      0
Site_Comments         39104
Sample_Comments       38403
Bleaching_Comments    38692
Length: 62, dtype: int64

In [42]:
# summary table function
pd.options.display.float_format = '{:,.2f}'.format
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values 
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    summ['first value'] = df.loc[0].values
    summ['second value'] = df.loc[1].values
    summ['third value'] = df.loc[2].values
    
    return summ

In [43]:
summary(df).style.background_gradient(cmap='YlOrBr')

data shape: (41361, 62)


,data type,#missing,%missing,#unique,min,max,average,standard_deviation,first value,second value,third value
Site_ID,int64,0,0.000000,12702,1.000000,1000060.000000,74558.164793,252041.802612,2501,3467,1794
Sample_ID,int64,0,0.000000,27005,9623.000000,10331713.000000,10128797.707817,1373151.378025,10324336,10324754,10323866
Data_Source,object,0,0.000000,9,nan,nan,nan,nan,Donner,Donner,Donner
Latitude_Degrees,float64,0,0.000000,11066,-30.262500,36.750000,7.558085,15.732185,23.163000,-17.575000,18.369000
Longitude_Degrees,float64,0,0.000000,11025,-179.974300,179.964500,34.966127,103.404598,-82.526000,-149.783300,-64.564000
Ocean_Name,object,0,0.000000,5,nan,nan,nan,nan,Atlantic,Pacific,Atlantic
Reef_ID,object,12540,30.318416,5354,nan,nan,nan,nan,nan,nan,nan
Realm_Name,object,0,0.000000,9,nan,nan,nan,nan,Tropical Atlantic,Eastern Indo-Pacific,Tropical Atlantic
Ecoregion_Name,object,3,0.007253,114,nan,nan,nan,nan,Cuba and Cayman Islands,Society Islands French Polynesia,Hispaniola Puerto Rico and Lesser Antilles
Country_Name,object,1,0.002418,90,nan,nan,nan,nan,Cuba,French Polynesia,United Kingdom


In [46]:
df['Bleaching_Level']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
41356    NaN
41357    NaN
41358    NaN
41359    NaN
41360    NaN
Name: Bleaching_Level, Length: 41361, dtype: object

## Processing

### Convert Data types

does something to bleaching level - fix it
errors = coerce is the problem

In [23]:
# Convert date columns to datetime
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    print("\n' Date' column converted to datetime.")

# List of numerical columns to convert
numerical_columns = [
    'Latitude_Degrees', 'Longitude_Degrees', 'Distance_to_Shore', 'Turbidity',
    'Cyclone_Frequency', 'Depth_m', 'Percent_Cover', 'Bleaching_Level',
    'Percent_Bleaching', 'ClimSST', 'Temperature_Kelvin', 'Temperature_Mean',
    'Temperature_Minimum', 'Temperature_Maximum', 'Temperature_Kelvin_Standard_Deviation',
    'Windspeed', 'SSTA', 'SSTA_Standard_Deviation', 'SSTA_Mean', 'SSTA_Minimum',
    'SSTA_Maximum', 'SSTA_Frequency', 'SSTA_Frequency_Standard_Deviation',
    'SSTA_FrequencyMax', 'SSTA_FrequencyMean', 'SSTA_DHW',
    'SSTA_DHW_Standard_Deviation', 'SSTA_DHWMax', 'SSTA_DHWMean',
    'TSA', 'TSA_Standard_Deviation', 'TSA_Minimum', 'TSA_Maximum',
    'TSA_Mean', 'TSA_Frequency', 'TSA_Frequency_Standard_Deviation',
    'TSA_FrequencyMax', 'TSA_FrequencyMean', 'TSA_DHW',
    'TSA_DHW_Standard_Deviation', 'TSA_DHWMax', 'TSA_DHWMean'
]

# Convert columns to numeric, coercing errors to NaN
for col in numerical_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nNumerical columns have been converted to appropriate data types.")



' Date' column converted to datetime.

Numerical columns have been converted to appropriate data types.


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41361 entries, 0 to 41360
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Site_ID                                41361 non-null  int64  
 1   Sample_ID                              41361 non-null  int64  
 2   Data_Source                            41361 non-null  object 
 3   Latitude_Degrees                       41361 non-null  float64
 4   Longitude_Degrees                      41361 non-null  float64
 5   Ocean_Name                             41361 non-null  object 
 6   Reef_ID                                28821 non-null  object 
 7   Realm_Name                             41361 non-null  object 
 8   Ecoregion_Name                         41358 non-null  object 
 9   Country_Name                           41360 non-null  object 
 10  State_Island_Province_Name             41262 non-null  object 
 11  Ci

#### Dropping unimportant columns

Reef_id, Substrate_Name, Site_Id, Sample_Id, Data_Source, Site_Comments, Sample_Comments, Bleaching_Comments

In [29]:
df.drop(['Reef_ID', 'Substrate_Name', 'Site_ID', 'Sample_ID', 'Data_Source', 'Site_Comments', 'Sample_Comments', 'Bleaching_Comments'], 
                axis = 1, inplace = True)

KeyError: "['Reef_ID', 'Substrate_Name', 'Site_ID', 'Sample_ID', 'Data_Source', 'Site_Comments', 'Sample_Comments', 'Bleaching_Comments'] not found in axis"

In [30]:
df

,Latitude_Degrees,Longitude_Degrees,Ocean_Name,Realm_Name,Ecoregion_Name,Country_Name,State_Island_Province_Name,City_Town_Name,Site_Name,Distance_to_Shore,...,TSA_Mean,TSA_Frequency,TSA_Frequency_Standard_Deviation,TSA_FrequencyMax,TSA_FrequencyMean,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean,Date
0,23.16,-82.53,Atlantic,Tropical Atlantic,Cuba and Cayman Islands,Cuba,Havana,Havana,Puntilla,"8,519.23",...,-2.17,0.00,1.09,5.00,0.00,0.00,0.74,7.25,0.18,2005-09-15
1,-17.57,-149.78,Pacific,Eastern Indo-Pacific,Society Islands French Polynesia,French Polynesia,Society Islands,Moorea,NaN,"1,431.62",...,-1.26,0.25,0.93,4.00,0.00,0.26,0.67,4.65,0.19,1991-03-15
2,18.37,-64.56,Atlantic,Tropical Atlantic,Hispaniola Puerto Rico and Lesser Antilles,United Kingdom,British Virgin Islands,Peter Island,Coral Gardens,182.33,...,-1.49,7.00,1.31,7.00,0.00,0.00,1.04,11.66,0.26,2006-01-15
3,17.76,-64.57,Atlantic,Tropical Atlantic,Hispaniola Puerto Rico and Lesser Antilles,United States,US Virgin Islands,St. Croix,3219,313.13,...,-1.49,3.00,0.94,4.00,0.00,0.00,0.75,5.64,0.20,2006-04-15
4,17.77,-64.58,Atlantic,Tropical Atlantic,Hispaniola Puerto Rico and Lesser Antilles,United States,US Virgin Islands,St. Croix,3194,792.00,...,-1.50,3.00,1.33,5.00,0.00,0.00,0.92,6.89,0.25,2006-04-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41356,-8.37,116.08,Pacific,Central Indo-Pacific,Lesser Sunda Islands and Savu Sea,Indonesia,West Nusa Tenggara,North Lombok Regency,Statue Garden - South of Gili Air,91.31,...,-0.95,1.00,2.00,8.00,1.00,2.09,1.49,9.00,0.63,2019-05-28
41357,-8.35,116.05,Pacific,Central Indo-Pacific,Lesser Sunda Islands and Savu Sea,Indonesia,West Nusa Tenggara,North Lombok Regency,Meno Slope,129.08,...,-1.00,1.00,1.75,8.00,1.00,2.00,1.29,8.01,0.65,2019-05-16
41358,-8.35,116.05,Pacific,Central Indo-Pacific,Lesser Sunda Islands and Savu Sea,Indonesia,West Nusa Tenggara,North Lombok Regency,Meno Slope,129.08,...,-1.00,1.00,1.75,8.00,1.00,2.00,1.29,8.01,0.65,2019-05-16
41359,-8.34,116.06,Pacific,Central Indo-Pacific,Lesser Sunda Islands and Savu Sea,Indonesia,West Nusa Tenggara,North Lombok Regency,Turtle City,209.17,...,-1.00,1.00,1.75,8.00,1.00,2.00,1.29,8.01,0.65,2019-05-29


In [32]:
df.to_csv('imp_cols.csv', encoding='utf-8', index=False)

In [33]:
summary(df).style.background_gradient(cmap='YlOrBr')

data shape: (41361, 54)


,data type,#missing,%missing,#unique,min,max,average,standard_deviation,first value,second value,third value
Latitude_Degrees,float64,0,0.000000,11066,-30.262500,36.750000,7.558085,15.732185,23.163000,-17.575000,18.369000
Longitude_Degrees,float64,0,0.000000,11025,-179.974300,179.964500,34.966127,103.404598,-82.526000,-149.783300,-64.564000
Ocean_Name,object,0,0.000000,5,nan,nan,nan,nan,Atlantic,Pacific,Atlantic
Realm_Name,object,0,0.000000,9,nan,nan,nan,nan,Tropical Atlantic,Eastern Indo-Pacific,Tropical Atlantic
Ecoregion_Name,object,3,0.007253,114,nan,nan,nan,nan,Cuba and Cayman Islands,Society Islands French Polynesia,Hispaniola Puerto Rico and Lesser Antilles
Country_Name,object,1,0.002418,90,nan,nan,nan,nan,Cuba,French Polynesia,United Kingdom
State_Island_Province_Name,object,99,0.239356,480,nan,nan,nan,nan,Havana,Society Islands,British Virgin Islands
City_Town_Name,object,1133,2.739295,1897,nan,nan,nan,nan,Havana,Moorea,Peter Island
Site_Name,object,34429,83.240250,2237,nan,nan,nan,nan,Puntilla,nan,Coral Gardens
Distance_to_Shore,float64,2,0.004835,12081,3.200000,299218.500000,3761.764707,13550.414805,8519.230000,1431.620000,182.330000


In [35]:
df['Bleaching_Level']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
41356   NaN
41357   NaN
41358   NaN
41359   NaN
41360   NaN
Name: Bleaching_Level, Length: 41361, dtype: float64